In [1]:
!pip install torch
!pip install transformers[torch]
!pip install accelerate==0.26.0
!pip install emoji

In [2]:
import pandas as pd
import numpy as np
import csv
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

W0506 19:05:10.778000 5328 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


# Preprocessing

In [4]:
df = pd.read_csv('C:/Users/ASUS/Desktop/Tugasakhir_M Naufal/Dataset/Data TA_LABEL.csv')
df['label_manual'] = df['label_manual'].fillna(0).astype(int)
df.head(10)

,textDisplay,label_manual
0,"Hutan Bojonegoro udah habis di tebang i,,dan d...",0
1,"Jangan khawatir deforestasi!!, kata pak presiden",1
2,"Assalamu&#39;alaikum, izin mengambil salah sat...",1
3,Saya tetap yakin kita terlahir dari nenek moya...,2
4,sekarang rasakan panasnya nggk seperti biasany...,1
5,Sbnr nya hutan milik siapa? Kok berani membaba...,1
6,"demi menyenangkan sahabat jokowi, si haji Is*m...",0
7,Mampus.....Suatu saat kalimantan bakalan gundul,0
8,Kalau orang hidup di kabupaten KKU berdampinga...,0
9,Kalimantan rusak itu akibat ulah pejabat dan o...,0


In [5]:
print(df.shape)

(857, 2)


In [6]:
import re
from bs4 import BeautifulSoup
import emoji

def clean_text(text):
    if isinstance(text, str):
        if re.search(r'<.*?>', text):
            text = BeautifulSoup(text, "html.parser").get_text() 

        text = emoji.replace_emoji(text, replace='')  
        text = re.sub(r'[^a-zA-Z\s]', '', text)  
        text = re.sub(r'\s+', ' ', text)  
        text = text.strip()  
    else:
        text = ""  
    
    return text

df['cleaned_text'] = df['textDisplay'].apply(clean_text)

df = df[df['cleaned_text'] != ""]

print(df[['cleaned_text', 'label_manual']].head(10))


                                        cleaned_text  label_manual
0  Hutan Bojonegoro udah habis di tebang idan di ...             0
1      Jangan khawatir deforestasi kata pak presiden             1
2  Assalamualaikum izin mengambil salah satu klip...             1
3  Saya tetap yakin kita terlahir dari nenek moya...             2
4  sekarang rasakan panasnya nggk seperti biasany...             1
5  Sbnr nya hutan milik siapa Kok berani membabal...             1
6  demi menyenangkan sahabat jokowi si haji Ism j...             0
7         MampusSuatu saat kalimantan bakalan gundul             0
8  Kalau orang hidup di kabupaten KKU berdampinga...             0
9  Kalimantan rusak itu akibat ulah pejabat dan o...             0


In [7]:
output_path = 'C:/Users/ASUS/Desktop/Tugasakhir_M Naufal/Dataset/dataset_preprocessing.csv'
df.to_csv(output_path, index=False)

In [8]:
df = pd.read_csv('C:/Users/ASUS/Desktop/Tugasakhir_M Naufal/Dataset/dataset_preprocessing.csv')


normalization_dict = {
    'sbnrnya': 'sebenarnya',
    'sbnr': 'sebenarnya',
    'ak': 'aku',
    'gk': 'gak',
    'gak': 'tidak',
    'yg': 'yang',
    'klo': 'kalau',
    'kl': 'kalau',
    'bg': 'banget',
    'gt': 'gitu',
    'gtu': 'gitu',
    'jgn': 'jangan',
    'gpp': 'gapapa',
    'jd': 'jadi',
    'org': 'orang',
    'tak': 'tidak',
    'tgung': 'tanggung',
    'sdh': 'sudah',
    'dah': 'sudah',
    'udh': 'sudah',
    'dh': 'sudah',
    'bgt': 'banget',
    'tmn': 'teman',
    'br': 'baru',
    'lbh': 'lebih',
    'krn': 'karena',
    'SDA': 'sumber daya alam',
    'seblm': 'sebelum',
    'skrg': 'sekarang',
    'skr': 'sekarang',
    'gedhe': 'gede',
    'ngk': 'engga',
    'kn': 'kan',
    'byk': 'banyak',
    'bnyk': 'banyak',
    'gw': 'aku',
    'gue': 'aku',
    'gwk': 'aku',
    'loe': 'kamu',
    'lu': 'kamu',
    'lo': 'kamu',
    'elu': 'kamu',
    'elo': 'kamu',
    'napa': 'kenapa',
    'knp': 'kenapa',
    'mgkn': 'mungkin',
    'gmn': 'gimana',
    'bkn': 'bukan',
    'bs': 'bisa',
    'bisa2': 'bisa-bisa',
    'diaa': 'dia',
    'bgtu': 'begitu',
    'btw': 'ngomong-ngomong',
    'dongg': 'dong',
    'gituu': 'gitu',
    'ntr': 'nanti',
    'msh': 'masih',
    'mauu': 'mau',
    'yakinbgt': 'yakin banget',
    'sampe': 'sampai',
    'mo': 'mau',
    'kek': 'seperti',
    'kyk': 'seperti',
    'ky': 'kayak',
    'ga': 'tidak',
    'enggak': 'tidak',
    'engga': 'tidak',
    'nggak': 'tidak',
    'ngga': 'tidak',
    'tdk': 'tidak',
    'okee': 'oke',
    'okeeee': 'oke',
    'gtw': 'gatau',
    'gatau': 'tidak tahu',
    'gaada': 'tidak ada',
    'gakada': 'tidak ada',
    'gbs': 'tidak bisa',
    'gabisa': 'tidak bisa',
    'gajadi': 'tidak jadi',
    'gapapa': 'tidak apa-apa',
    'gausah': 'tidak usah',
    'udahlah': 'sudah lah',
    'kmn': 'kemana',
    'kmna': 'kemana',
    'sma': 'sama',
    'mls': 'malas',
    'mlas': 'malas',
    'trs2': 'terus-terus',
    'trus2': 'terus-terus',
    'mager': 'malas gerak',
    'drpd': 'daripada',
    'abis': 'habis',
    'mg': 'minggu',
    'biar': 'agar',
    'ampun': 'tidak sanggup',
    'parahh': 'parah',
    'zz': '',
    'zzz': '',
    'wkwk': '',
    'wkwkwk': '',
    'lol': '',
    'hehe': '',
    'haha': '',
    'huhu': '',
    'yaa': 'iya',
    'iyaaa': 'iya',
    'iiya': 'iya',
    'okei': 'oke',
    'okehh': 'oke',
    'heemm': '',
    'tp': 'tapi',
    'dmn': 'dimana',
    'aja': 'saja',
    'dg': 'dengan',
    'dr': 'dari',
    'dpt': 'dapat',
    'karna': 'karena',
    'blm': 'belum',
    'skrng': 'sekarang',
    'ntar': 'nanti',
    'kpn': 'kapan',
    'trs': 'terus',
    'pdhl': 'padahal',
    'kmrn': 'kemarin',
    'km': 'kamu',
    'lg': 'lagi',
    'mknya': 'makanya',
    'trus': 'terus',
    'donk': 'dong',
    'sih': '',
    'deh': '',
    'nya': '',
    'kok': '',
    'hdh': '',
    'cuma': 'hanya',
    'cuman': 'hanya',
    'y': 'ya',
    'thx': 'terima kasih',
    'thnk': 'terima kasih',
    'makasih': 'terima kasih',
    'thanks': 'terima kasih',
    'sm': 'sama',
    'sama2': 'sama-sama',
    'sama2': 'sama-sama',
    'idan': 'dan',
    'pd': 'pada',
    'ya': 'iya',
    'nggk ': 'engga'
}

def normalize_text(text):
    text = text.lower()  
    words = text.split()  
    normalized_words = [normalization_dict.get(word, word) for word in words]  
    return ' '.join(normalized_words)  

df['normalized_text'] = df['cleaned_text'].apply(normalize_text)
print(df[['cleaned_text', 'normalized_text']].head(10))

output_path = 'C:/Users/ASUS/Desktop/Tugasakhir_M Naufal/Dataset/cleaned_text.csv'
df[['normalized_text', 'label_manual']].to_csv(output_path, index=False)


                                        cleaned_text  \
0  Hutan Bojonegoro udah habis di tebang idan di ...   
1      Jangan khawatir deforestasi kata pak presiden   
2  Assalamualaikum izin mengambil salah satu klip...   
3  Saya tetap yakin kita terlahir dari nenek moya...   
4  sekarang rasakan panasnya nggk seperti biasany...   
5  Sbnr nya hutan milik siapa Kok berani membabal...   
6  demi menyenangkan sahabat jokowi si haji Ism j...   
7         MampusSuatu saat kalimantan bakalan gundul   
8  Kalau orang hidup di kabupaten KKU berdampinga...   
9  Kalimantan rusak itu akibat ulah pejabat dan o...   

                                     normalized_text  
0  hutan bojonegoro udah habis di tebang dan di t...  
1      jangan khawatir deforestasi kata pak presiden  
2  assalamualaikum izin mengambil salah satu klip...  
3  saya tetap yakin kita terlahir dari nenek moya...  
4  sekarang rasakan panasnya nggk seperti biasany...  
5  sebenarnya  hutan milik siapa  berani membabal... 

Tokenisasi 

In [10]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd

#  tokenizer untuk IndoBERT
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")


df = pd.read_csv('C:/Users/ASUS/Desktop/Tugasakhir_M Naufal/Dataset/cleaned_text.csv')

# tokenisasi, padding, dan truncation
def tokenize_text(text):
    if isinstance(text, str) and text.strip() != "":
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,  
            max_length=128,           
            padding='max_length',     
            truncation=True,          
            return_tensors='pt'       
        )
        input_ids = encoding['input_ids'].flatten().tolist()  
        tokens = tokenizer.convert_ids_to_tokens(input_ids)  
        return tokens, input_ids
    else:
        return [], [] 

df['tokens'] = df['normalized_text'].apply(lambda x: tokenize_text(x)[0])
df['tokenized_text'] = df['normalized_text'].apply(lambda x: tokenize_text(x)[1])

print(df[['tokens', 'tokenized_text']].head(10))

output_path = 'C:/Users/ASUS/Desktop/Tugasakhir_M Naufal/Dataset/dataset_tokenized_indobert.csv'
df.to_csv(output_path, index=False)

C:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


                                              tokens  \
0  [[CLS], hutan, bojonegoro, udah, habis, di, te...   
1  [[CLS], jangan, khawatir, def, ##ores, ##tasi,...   
2  [[CLS], assalamualaikum, izin, mengambil, sala...   
3  [[CLS], saya, tetap, yakin, kita, terlahir, da...   
4  [[CLS], sekarang, rasakan, panasnya, ng, ##g, ...   
5  [[CLS], sebenarnya, hutan, milik, siapa, beran...   
6  [[CLS], demi, menyenangkan, sahabat, jokowi, s...   
7  [[CLS], mampu, ##ss, ##uatu, saat, kalimantan,...   
8  [[CLS], kalau, orang, hidup, di, kabupaten, kk...   
9  [[CLS], kalimantan, rusak, itu, akibat, ulah, ...   

                                      tokenized_text  
0  [3, 3283, 15125, 9988, 5594, 1485, 5491, 1480,...  
1  [3, 3132, 6348, 4747, 18077, 4823, 1951, 2458,...  
2  [3, 26460, 5297, 2948, 1911, 1713, 13777, 1485...  
3  [3, 1731, 2261, 4455, 1732, 14368, 1542, 6551,...  
4  [3, 2338, 14220, 26061, 2870, 938, 942, 1730, ...  
5  [3, 2923, 3283, 3139, 3476, 5115, 1672, 7617, ... 

Attention Mask

In [12]:
def create_attention_mask(input_ids):
    attention_mask = [1 if token_id != tokenizer.pad_token_id else 0 for token_id in input_ids]
    return attention_mask

df['attention_mask'] = df['tokenized_text'].apply(lambda x: create_attention_mask(x))

Konversi ke Tensors

In [14]:
import torch

def convert_to_tensor(input_ids, attention_mask):
    input_ids_tensor = torch.tensor(input_ids)
    attention_mask_tensor = torch.tensor(attention_mask)
    return input_ids_tensor, attention_mask_tensor

df['input_ids_tensor'], df['attention_mask_tensor'] = zip(*df.apply(lambda row: convert_to_tensor(row['tokenized_text'], row['attention_mask']), axis=1))


In [15]:
from sklearn.model_selection import train_test_split

#  data menjadi train dan test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

#  data train menjadi train dan validation
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)


print(f"Jumlah data untuk training set: {len(train_df)}")
print(f"Jumlah data untuk validation set: {len(val_df)}")
print(f"Jumlah data untuk test set: {len(test_df)}")


Jumlah data untuk training set: 609
Jumlah data untuk validation set: 68
Jumlah data untuk test set: 170


In [16]:
from torch import nn

# Menghitung bobot kelas berdasarkan distribusi label
class_weights = [1 / df['label_manual'].value_counts().get(i, 1) for i in range(3)]

# Konversi ke tensor
class_weights_tensor = torch.tensor(class_weights)

loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)

In [17]:
# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Prepare datasets
train_dataset = SentimentDataset(
    torch.stack(train_df['input_ids_tensor'].tolist()),
    torch.stack(train_df['attention_mask_tensor'].tolist()),
    torch.tensor(train_df['label_manual'].values)
)

val_dataset = SentimentDataset(
    torch.stack(val_df['input_ids_tensor'].tolist()),
    torch.stack(val_df['attention_mask_tensor'].tolist()),
    torch.tensor(val_df['label_manual'].values)
)

test_dataset = SentimentDataset(
    torch.stack(test_df['input_ids_tensor'].tolist()),
    torch.stack(test_df['attention_mask_tensor'].tolist()),
    torch.tensor(test_df['label_manual'].values)
)

In [18]:
# Custom BERT classifier with an optional hidden layer
class CustomBertClassifier(nn.Module):
    def __init__(self, base_model, num_labels=3, hidden_layer=False):
        super().__init__()
        self.bert = base_model
        self.num_labels = num_labels
        if hidden_layer:
            self.hidden = nn.Linear(self.bert.config.hidden_size, 128)
            self.classifier = nn.Linear(128, num_labels)
        else:
            self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0, :]  # CLS token
        if hasattr(self, 'hidden'):
            hidden_state = self.hidden(hidden_state)
        logits = self.classifier(hidden_state)
        return logits

In [19]:
use_hidden_layer = True
model = CustomBertClassifier(base_model=AutoModel.from_pretrained("indolem/indobert-base-uncased"), hidden_layer=use_hidden_layer)


C:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
# Freezing the BERT backbone layers
for param in model.bert.parameters():
    param.requires_grad = False

# Only the classifier layer will be updated
for param in model.classifier.parameters():
    param.requires_grad = True

In [21]:
model = BertForSequenceClassification.from_pretrained(
    "indolem/indobert-base-uncased", 
    num_labels=3  
)

# Optimizer dan scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# CSV logging function
def log_metrics_to_csv(metrics, epoch, filename='metrics_log.csv'):
    file_exists = False
    try:
        with open(filename, 'r') as file:
            file_exists = True
    except FileNotFoundError:
        pass

    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['epoch', 'accuracy', 'precision', 'recall', 'f1'])

        writer.writerow([epoch, metrics['accuracy'], metrics['precision'], metrics['recall'], metrics['f1']])

In [24]:

def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)  
    metrics = {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='weighted'),
        'recall': recall_score(labels, preds, average='weighted'),
        'f1': f1_score(labels, preds, average='weighted')
    }

    log_metrics_to_csv(metrics, trainer.state.epoch)  
    return metrics

In [25]:
import time

def measure_inference_time(model, data_point, num_runs=10):
    total_time = 0
    for _ in range(num_runs):
        start_time = time.time()
        with torch.no_grad():
            input_ids = data_point['input_ids'].unsqueeze(0)  
            attention_mask = data_point['attention_mask'].unsqueeze(0)  
            _ = model(input_ids, attention_mask=attention_mask)
        total_time += time.time() - start_time
    average_time = total_time / num_runs
    return average_time

sample_data_point = val_dataset[0]  
avg_inference_time = measure_inference_time(model, sample_data_point)
print(f"Rata-rata waktu komputasi untuk satu data point: {avg_inference_time:.4f} detik")

Rata-rata waktu komputasi untuk satu data point: 0.1711 detik


In [26]:
training_args = TrainingArguments(
    output_dir='./results',           
    num_train_epochs=20,              
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=64,    
    warmup_steps=500,                 
    weight_decay=0.01,                
    logging_dir='./logs',             
    logging_steps=1,                  
    evaluation_strategy="epoch",      
    save_strategy="epoch",            
    load_best_model_at_end=True,      
    lr_scheduler_type="linear",       
)

# Trainer setup
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,            
    compute_metrics=compute_metrics,     
    optimizers=(optimizer, None),        
)

In [27]:
# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate(test_dataset)

# Save the model
trainer.save_model('./final_model')

# Save the tokenizer
tokenizer.save_pretrained('./final_model')

C:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.220500,1.084343,0.352941,0.532428,0.352941,0.344254
2,0.841300,1.078685,0.352941,0.512374,0.352941,0.351884
3,1.132400,1.069519,0.338235,0.461172,0.338235,0.338817
4,1.074700,1.057336,0.338235,0.452378,0.338235,0.346081
5,1.057000,1.042272,0.382353,0.442857,0.382353,0.400735
6,1.060300,1.025155,0.455882,0.468665,0.455882,0.459913
7,1.084400,1.006886,0.455882,0.451184,0.455882,0.452705
8,1.335800,0.987825,0.500000,0.444118,0.500000,0.468173
9,0.889500,0.968816,0.558824,0.466243,0.558824,0.498102
10,0.949200,0.949799,0.558824,0.442191,0.558824,0.483922


C:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Anaconda\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then de

C:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('./final_model\\tokenizer_config.json',
 './final_model\\special_tokens_map.json',
 './final_model\\vocab.txt',
 './final_model\\added_tokens.json',
 './final_model\\tokenizer.json')